<a href="https://colab.research.google.com/github/WillerLucoles/Spark_Bigdata/blob/main/WillerLucoles_trabalho_big_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Upload do arquivo `imdb-reviews-pt-br.csv` para dentro do Google Colab

In [1]:
!wget https://raw.githubusercontent.com/N-CPUninter/Big_Data/main/data/imdb-reviews-pt-br.zip -O imdb-reviews-pt-br.zip
!unzip imdb-reviews-pt-br.zip
!rm imdb-reviews-pt-br.zip

--2024-10-03 22:35:53--  https://raw.githubusercontent.com/N-CPUninter/Big_Data/main/data/imdb-reviews-pt-br.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49549692 (47M) [application/zip]
Saving to: ‘imdb-reviews-pt-br.zip’

imdb-reviews-pt-br. 100%[===================>]  47.25M   218MB/s    in 0.2s    

2024-10-03 22:35:53 (218 MB/s) - ‘imdb-reviews-pt-br.zip’ saved [49549692/49549692]

Archive:  imdb-reviews-pt-br.zip
  inflating: imdb-reviews-pt-br.csv  


## Instalação manual das dependências para uso do pyspark no Google Colab

In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=a957e76161bbed485f2a35759fabbc0184c1e438b6a943ca1fb00292615afedc
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


## Importar, instanciar e criar a SparkSession

In [3]:
from pyspark.sql import SparkSession

appName = "PySpark Trabalho de Big Data"
master = "local"

spark = SparkSession.builder.appName(appName).master(master).getOrCreate()

## Criar spark dataframe do CSV utilizando o método read.csv do spark

In [4]:
imdb_df = spark.read.csv('imdb-reviews-pt-br.csv',
                         header=True,
                         quote="\"",
                         escape="\"",
                         encoding="UTF-8")

# Questão 1

## Criar funções de MAP:
- Criar função para mapear o "sentiment" como chave e o "id" como valor do tipo inteiro

In [5]:
def map1(row):
    if row['sentiment'] == 'neg':
        return (row['sentiment'], int(row['id']))
    return None


## Cria funções de REDUCE:

- Criar função de reduce para somar os IDs por "sentiment".

In [6]:
def reduceByKey1(x, y):
    return x + y


## Aplicação do map/reduce e visualização do resultado

In [7]:

rdd = imdb_df.rdd
mapped_rdd = rdd.map(map1).filter(lambda x: x is not None)
result = mapped_rdd.reduceByKey(reduceByKey1).collect()

print("Willer Lucas - RU: 1351633")
print(f"Resultado da soma dos IDs para resenhas negativas: {result}")



Willer Lucas - RU: 1351633
Resultado da soma dos IDs para resenhas negativas: [('neg', 459568555)]


# Questão 2:

## Criar funções de MAP:
- Criar função para mapear o "sentiment" como chave e uma tupla com a soma das palavras de cada texto como valor.

In [10]:
def map2(row):
    if row['sentiment'] == 'neg':
        num_words_pt = len(row['text_pt'].split()) if row['text_pt'] else 0
        num_words_en = len(row['text_en'].split()) if row['text_en'] else 0
        return (row['sentiment'], (num_words_pt, num_words_en))
    return None


## Cria funções de REDUCE:

- Criar função de reduce para somar o numero de palavras de cada texto português e inglês por "sentiment".

In [11]:
def reduceByKey2(x, y):
    return (x[0] + y[0], x[1] + y[1])


## Aplicação do map/reduce e visualização do resultado

1. Aplicar o map/reduce no seu dataframe spark e realizar o collect() ao final
2. Selecionar os dados referentes aos textos negativos para realizar a subtração.
3. Realizar a subtração das contagens de palavras dos textos negativos para obter o resultado final

In [12]:
rdd = imdb_df.rdd
mapped_rdd = rdd.map(map2).filter(lambda x: x is not None)
result = mapped_rdd.reduceByKey(reduceByKey2).collect()

for sentiment, (total_words_pt, total_words_en) in result:
    print(f"Total de palavras em português: {total_words_pt}")
    print(f"Total de palavras em inglês: {total_words_en}")
    print(f"Diferença (português - inglês): {total_words_pt - total_words_en}")
print("Willer Lucas - RU: 1351633")


Total de palavras em português: 5455273
Total de palavras em inglês: 5400324
Diferença (português - inglês): 54949
Willer Lucas - RU: 1351633
